In [1]:
import os
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from tqdm.notebook import tqdm

## Original Parameter values

## ODE

In [2]:
def f(x, t):
    
    #times of new restrictions
    timeslot_1 = 589 - 549
    timeslot_2 = timeslot_1 + 689-589
    
    if t<(timeslot_1):
            ρ = ρ1 
            σ = σ1 
            τ = τ1 
    elif t>=timeslot_1 and t < timeslot_2:
            ρ = ρ2 
            σ = σ2 
            τ = τ2
    else:
            ρ = ρ3
            σ = σ3
            τ = τ3
        
    S,E,Q_1,Q_2,I,V,    L,U,D,R,H = x
    
    N = S + E + Q_1 + Q_2 + I + V
    
    Q = Q_1 + Q_2
    
    R_G = a*R/(1+a*R)

    η_1 = η * 0.897 * (S/(S+V))
    η_2 = η * 0.103
    η_3 = η * 0.897 * (V/(S+V))

    dS = -β * I/(N-Q) * S - ψ*( 1 - (R + τ*(U))/N + D/N ) * S - ρ * (1- (R + τ * (U))/N) * S + η_1 * Q_1
    dE = β * I/(N-Q) * S + β * δ * I/(N-Q) * V - ρ * ( 1- (R + τ * (U))/N) * E - γ * E
    dQ_1 = ρ * ( 1- (R + τ * (U))/N) * S + ρ * ( 1- (R + τ * (U))/N) * E + ρ * ( 1- (R + τ * (U))/N) * V - η_1 * Q_1 - η_2 * Q_1 - η_3 * Q_1
    dQ_2 = σ * ( 1- R/N) * I + η_2 * Q_1 - r_2 * Q_2
    dI = γ * E - σ * ( 1- R/N) * I - r_1 * I
    dV = r_1 * I + r_2 * Q_2 + η_3 * Q_1 + ψ * ( 1- (R + τ * (U))/N + D/N) * S - β * δ * I/(N-Q) * V - ρ * ( 1- (R + τ * (U))/N) * V
    
    dL = -α * R/N * L*(1-(I+Q_2)/N) - α * U/N * L - α * D/N * L*(1+(I+Q_2)/N) + ϕ * R + ϕ * H

    dU = α * U/N * L-  χ* R_G/N * U *(1-(I+Q_2)/N) - χ * D/N * U*(1+(I+Q_2)/N) - λ * U
    dD = α * D/N * L*(1+(I+Q_2)/N) + χ * D/N * U*(1+(I+Q_2)/N) - λ * D 
    dR = α * R/N * L* (1-(I+Q_2)/N) + χ * R_G/N * U*(1-(I+Q_2)/N) - ϕ * R 
    dH = λ * D + λ * U - ϕ * H

    return [dS,dE,dQ_1,dQ_2,dI,dV,dL,dU,dD,dR,dH]

## Initial values

In [3]:
U = 140000
D = 1400000
R = 350000
H = 0

Q_2 = 3649881-3642450 #recovered from 12.07.-01.07
V = 34715161


In [4]:
#Epidemic model
β = 0.4
r_1 = 0.1
r_2 = 1/12
γ = 1/4
δ = 0.28
ψ = 0.00199

#Opinion model
α = 0.071
ϕ = 1.9685e-5
χ = 0.0031
λ = 4.2972e-5
η = 1/2.5
a = 0.8

ρ1 = 1.959769532254296287e-04
σ1 = 1.524108160070437035e-01
τ1 = 2.954975435572986253e-01
ρ2 = 1.282332136525898836e-03
σ2 = 5.499123541780154278e-02
τ2 = 5.705622081879080376e-01
ρ3 = 7.219319175577609826e-06
σ3 = 1.823169505631421772e-01
τ3 = 1.327866618363038320e-04

E = 1.711464626962858119e+03
Q_1 = 9.708158050998896215e+03
I = 9.788464694773647352e+03

In [5]:
params = np.array([β,r_1,r_2,γ,δ,ψ,α,ϕ,χ,λ,η,a,ρ1,σ1,τ1,ρ2,σ2,τ2,ρ3,σ3,τ3,E,Q_1,Q_2,I,V,U,D,R])
string_params = ['β','r_1','r_2','γ','δ','ψ','α','ϕ','χ','λ',
                 'η','a','ρ1','σ1','τ1','ρ2','σ2','τ2','ρ3','σ3','τ3','E','Q_1','Q_2','I','V','U','D','R']

In [6]:
S = 70000000 - E - Q_1 - Q_2 - I - V
L = 70000000 - D - R - H - U

x0 = [S, E, Q_1, Q_2, I, V,    #epidemics
      L, U, D, R, H]  #opinion

ts = np.linspace(0,183, 183)

In [7]:
original = odeint(f,x0,ts)

## Repeatedly Solve ODE with perturbed Parameters

In [8]:
def perturb_params(n, original_params, scale):
    
    solutions = []
    ts = np.linspace(0,183, 183)

    for i in tqdm(range(n), leave = False):
        perturbed_params = np.random.uniform( (1-scale)*original_params, (1+scale)*original_params)
        
        β,r_1,r_2,γ,δ,ψ,α,ϕ,χ,λ,η,a,ρ1,σ1,τ1,ρ2,σ2,τ2,ρ3,σ3,τ3,E,Q_1,Q_2,I,V,U,D,R = perturbed_params
        S = 70000000 - E - Q_1 - Q_2 - I - V
        L = 70000000 - D - R - H - U
        
        x0 = [S, E, Q_1, Q_2, I, V,    #epidemics
              L, U, D, R, H]  #opinion

        #define function again to get new params
        def f(x, t):
            #times of new restrictions
            timeslot_1 = 589 - 549
            timeslot_2 = timeslot_1 + 689-589
            timeslot_3 = timeslot_1+timeslot_2 + 703-689

            if t<(timeslot_1):
                ρ = ρ1
                σ = σ1
                τ = τ1
            elif t>=timeslot_1 and t < timeslot_2:
                ρ = ρ2
                σ = σ2
                τ = τ2
            else:
                ρ = ρ3
                σ = σ3
                τ = τ3

            S,E,Q_1,Q_2,I,V,    L,U,D,R,H = x

            N = S + E + Q_1 + Q_2 + I + V

            Q = Q_1 + Q_2

            R_G = a*R/(1+a*R)

            η_1 = η * 0.897 * (S/(S+V))
            η_2 = η * 0.103
            η_3 = η * 0.897 * (V/(S+V))

            dS = -β * I/(N-Q) * S - ψ*( 1 - (R + τ*(U))/N + D/N ) * S - ρ * (1- (R + τ * (U))/N) * S + η_1 * Q_1
            dE = β * I/(N-Q) * S + β * δ * I/(N-Q) * V - ρ * ( 1- (R + τ * (U))/N) * E - γ * E
            dQ_1 = ρ * ( 1- (R + τ * (U))/N) * S + ρ * ( 1- (R + τ * (U))/N) * E + ρ * ( 1- (R + τ * (U))/N) * V - η_1 * Q_1 - η_2 * Q_1 - η_3 * Q_1
            dQ_2 = σ * ( 1- R/N) * I + η_2 * Q_1 - r_2 * Q_2
            dI = γ * E - σ * ( 1- R/N) * I - r_1 * I
            dV = r_1 * I + r_2 * Q_2 + η_3 * Q_1 + ψ * ( 1- (R + τ * (U))/N + D/N) * S - β * δ * I/(N-Q) * V - ρ * ( 1- (R + τ * (U))/N) * V

            dL = -α * R/N * L*(1-(I+Q_2)/N) - α * U/N * L - α * D/N * L*(1+(I+Q_2)/N) + ϕ * R + ϕ * H

            dU = α * U/N * L-  χ* R_G/N * U *(1-(I+Q_2)/N) - χ * D/N * U*(1+(I+Q_2)/N) - λ * U
            dD = α * D/N * L*(1+(I+Q_2)/N) + χ * D/N * U*(1+(I+Q_2)/N) - λ * D
            dR = α * R/N * L* (1-(I+Q_2)/N) + χ * R_G/N * U*(1-(I+Q_2)/N) - ϕ * R
            dH = λ * D + λ * U - ϕ * H

            return [dS,dE,dQ_1,dQ_2,dI,dV,dL,dU,dD,dR,dH]
        solutions.append(odeint(f, x0, ts))
        
    return solutions

# Functions for finding the minimum and maximum deviations

In [9]:
def get_min_max_2(solutions):
    sol_arr = np.array(solutions)
    #get min/max solutions
    S_max = np.amax(sol_arr, axis=0)
    S_min = np.amin(sol_arr, axis=0)
    #get index of min/max solutions, index is taken from the first entry, if multiple are available
    S_max_index = np.argmax(sol_arr, axis=0)
    S_min_index = np.argmin(sol_arr, axis=0)

    return [S_max, S_min, S_max_index, S_min_index]


In [10]:
def find_max_min_diviation_param(sol_max, sol_min, original_curves, index_max, index_min, changed_params):
    save_max_param_per_compartment = []
    save_min_param_per_compartment = []

    k, n = sol_max.shape
    for i in range(n):
        ind_max = np.argmax(sol_max[:,i]-original_curves[:,i])
        ind_min = np.argmin(sol_min[:,i]-original_curves[:,i]) #takes the most negative one
        save_max_param_per_compartment.append(changed_params[index_max[ind_max,i]])
        save_min_param_per_compartment.append(changed_params[index_min[ind_min,i]])
    return [save_max_param_per_compartment, save_min_param_per_compartment]

In [11]:
def perturb_single_param_2(n, original_params, scale, index):

    mesh = np.linspace(original_params[index]*(1-scale), original_params[index]*(1+scale), n)
    solutions = []
    parameter_diviations = []
    ts = np.linspace(0,183, 183)

    for i in tqdm(range(n)):
        perturbed_params = np.copy(original_params)
        perturbed_params[index] = mesh[i]
        parameter_diviations.append(mesh[i])
        β,r_1,r_2,γ,δ,ψ,α,ϕ,χ,λ,η,a,ρ1,σ1,τ1,ρ2,σ2,τ2,ρ3,σ3,τ3,E,Q_1,Q_2,I,V,U,D,R = perturbed_params

        S = 70000000 - E - Q_1 - Q_2 - I - V
        L = 70000000 - D - R - H - U

        x0 = [S, E, Q_1, Q_2, I, V,    #epidemics
              L, U, D, R, H]   #opinion

        #define the function anew:
        def f(x, t):
            #times of new restrictions
            timeslot_1 = 589 - 549
            timeslot_2 = timeslot_1 + 689-589

            if t<(timeslot_1):
                ρ = ρ1
                σ = σ1
                τ = τ1
            elif t>=timeslot_1 and t < timeslot_2:
                ρ = ρ2
                σ = σ2
                τ = τ2
            else:
                ρ = ρ3
                σ = σ3
                τ = τ3

            S,E,Q_1,Q_2,I,V,    L,U,D,R,H = x

            N = S + E + Q_1 + Q_2 + I + V

            Q = Q_1 + Q_2

            R_G = a*R/(1+a*R)

            η_1 = η * 0.897 * (S/(S+V))
            η_2 = η * 0.103
            η_3 = η * 0.897 * (V/(S+V))

            dS = -β * I/(N-Q) * S - ψ*( 1 - (R + τ*(U))/N + D/N ) * S - ρ * (1- (R + τ * (U))/N) * S + η_1 * Q_1
            dE = β * I/(N-Q) * S + β * δ * I/(N-Q) * V - ρ * ( 1- (R + τ * (U))/N) * E - γ * E
            dQ_1 = ρ * ( 1- (R + τ * (U))/N) * S + ρ * ( 1- (R + τ * (U))/N) * E + ρ * ( 1- (R + τ * (U))/N) * V - η_1 * Q_1 - η_2 * Q_1 - η_3 * Q_1
            dQ_2 = σ * ( 1- R/N) * I + η_2 * Q_1 - r_2 * Q_2
            dI = γ * E - σ * ( 1- R/N) * I - r_1 * I
            dV = r_1 * I + r_2 * Q_2 + η_3 * Q_1 + ψ * ( 1- (R + τ * (U))/N + D/N) * S - β * δ * I/(N-Q) * V - ρ * ( 1- (R + τ * (U))/N) * V

            dL = -α * R/N * L*(1-(I+Q_2)/N) - α * U/N * L - α * D/N * L*(1+(I+Q_2)/N) + ϕ * R + ϕ * H

            dU = α * U/N * L-  χ* R_G/N * U *(1-(I+Q_2)/N) - χ * D/N * U*(1+(I+Q_2)/N) - λ * U
            dD = α * D/N * L*(1+(I+Q_2)/N) + χ * D/N * U*(1+(I+Q_2)/N) - λ * D
            dR = α * R/N * L* (1-(I+Q_2)/N) + χ * R_G/N * U*(1-(I+Q_2)/N) - ϕ * R
            dH = λ * D + λ * U - ϕ * H

            return [dS,dE,dQ_1,dQ_2,dI,dV,dL,dU,dD,dR,dH]



        solutions.append(odeint(f, x0, ts))

    return solutions, parameter_diviations

# Functions for an automatic plot

In [12]:
def plot_compartments_2(original_curves, perturbed_params,  single_solution, string_params, index, file_specific_range = False):
    compartment_names = ['S', 'E', '$Q_1$', '$Q_2$', 'I', 'V',    #epidemics
                         'L', 'U', 'D', 'R', 'H']
    S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_solution)
    max_params_per_comp, min_params_per_comp = find_max_min_diviation_param(S_max, S_min, original, S_max_index, S_min_index, perturbed_params)


    #Epidemie
    figure, axis = plt.subplots(6, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(6):
        axis[i].plot(ts, S_max[:,i], color = 'r')
        axis[i].plot(ts, original_curves[:,i], color = 'g')
        axis[i].plot(ts, S_min[:,i], color = 'b')
        axis[i].legend(['max ' + string_params[index] +': ' + str(max_params_per_comp[i]),
                        'original ' + string_params[index]+': '+ str(params[index]) ,
                        'min ' + string_params[index] +': '+ str(min_params_per_comp[i])])
        axis[i].set_title(compartment_names[i], fontsize = 16)

    if file_specific_range == True:
        figure.savefig('perturb_specific_range/epidemic_plot_' + string_params[index] +
                       '_perturbed.jpg',bbox_inches = 'tight', dpi=500)
    else:
        figure.savefig('perturbed_compartment_plots/epidemic_plot_' + string_params[index] +
                       '_perturbed.jpg',bbox_inches = 'tight', dpi=500)

    plt.close(figure)
    plt.close('all')

    #Opinion
    figure, axis = plt.subplots(5, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(6,11):
        axis[i-6].plot(ts, S_max[:,i], color = 'r')
        axis[i-6].plot(ts, original_curves[:,i], color = 'g')
        axis[i-6].plot(ts, S_min[:,i], color = 'b')
        axis[i-6].legend(['max ' + string_params[index] +': ' + str(max_params_per_comp[i]),
                        'original ' + string_params[index]+': '+ str(params[index]) ,
                        'min ' + string_params[index] +': '+ str(min_params_per_comp[i])])
        axis[i-6].set_title(compartment_names[i], fontsize = 16)

    if file_specific_range == True:
        figure.savefig('perturb_specific_range/opinion_plot_' + string_params[index] +
                       '_perturbed.jpg',bbox_inches = 'tight', dpi=500)
    else:
        figure.savefig('perturbed_compartment_plots/opinion_plot_' + string_params[index] +
                       '_perturbed.jpg', bbox_inches = 'tight', dpi=500)
    plt.close(figure)
    plt.close('all')

In [13]:
def plot_compartments_all(original_curves, single_solution, string_params, index, file_specific_range = False):
    compartment_names = ['S', 'E', '$Q_1$', '$Q_2$', 'I', 'V',    #epidemics
                         'L', 'U', 'D', 'R', 'H']

    S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_solution)

    #Epidemie
    figure, axis = plt.subplots(6, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(6):
        axis[i].plot(ts, S_max[:,i], color = 'r')
        axis[i].plot(ts, original_curves[:,i], color = 'g')
        axis[i].plot(ts, S_min[:,i], color = 'b')
        axis[i].legend(['max','original','min'])
        axis[i].set_title(compartment_names[i], fontsize = 16)

    figure.savefig('perturbed_compartment_plots/epidemic_plot_' + string_params[index] +
                   '_perturbed.jpg', bbox_inches='tight', dpi=500)
    plt.close(figure)
    plt.close('all')

    #Opinion
    figure, axis = plt.subplots(5, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(6,11):
        axis[i-6].plot(ts, S_max[:,i], color = 'r')
        axis[i-6].plot(ts, original_curves[:,i], color = 'g')
        axis[i-6].plot(ts, S_min[:,i], color = 'b')
        axis[i-6].legend(['max','original','min'])
        axis[i-6].set_title(compartment_names[i], fontsize = 16)

    figure.savefig('perturbed_compartment_plots/opinion_plot_' + string_params[index] +
                   '_perturbed.jpg', bbox_inches='tight', dpi=500)
    plt.close(figure)
    plt.close('all')


    #Only for the report

    plt.figure(figsize=(8.46, 4.7))
    plt.plot(ts, S_max[:,3], color = 'r')
    plt.plot(ts, original_curves[:,3], color = 'g')
    plt.plot(ts, S_min[:,i], color = 'b')
    plt.xlabel("t in days")
    plt.title("Uncertainty of compartment $Q_2$",fontsize = 16)
    plt.legend(['max','original','min'])
    plt.savefig('perturbed_compartment_plots/Simulation_uncertainty_all_perturbed_Q_2.jpg', bbox_inches = 'tight', dpi = 500)
    plt.close(figure)
    plt.close('all')

# Functions for calculating and ordering the two uncertainty measures

In [14]:
'''
The uncertainty measure in this function is defined as follows:
- the uncertainty is calculated by adding up the difference between the maximal and minimal values of each curve
- here the maximum deviation between all compartments and one compartment alone is calculated

Input:
- list_of_matrices, where len(list_of_matrices) = #Parameter and each matrix has the dimension D = 180x11, t x #compartments
- list_of_matrices already consists of the difference between the largest and smalles diviation:  S_max - Smin

- list_of_parameters is a list of strings with the parameters

Output:
- largest_diviation_param: list of parameters which caused the largest diviation in the sum over all timepoints t
'''

def find_most_uncertain_parameter(list_of_matrices,list_of_parameters):
    n = len(list_of_matrices) #this corresponds to the number of parameters


    #Calculate uncertainty by adding up all differences of the min and max curves
    vector_rowSum = []
    sum_all_compartments = []
    for i in range(n):
        A = np.matrix(np.copy(list_of_matrices[i]))
        v = A.sum(axis=0)
        deviation_all_compartments = np.sum(v)
        vector_rowSum.append(deviation_all_compartments)
        sum_all_compartments.append(v)
    #we want a list of all parameters and their uncertainty value
    ordered_by_uncertainty = []
    #if the largest value is recorded, then the value is set to -1, to get the second biggest value.
    for i in range(len(vector_rowSum)):
        k = np.argmax(vector_rowSum)
        value, name = vector_rowSum[k]/183, list_of_parameters[k]
        ordered_by_uncertainty.append([name, value])
        #the second largest. All values in these matrices are positive, so we simply put it to -1
        vector_rowSum[k] = -1


    return ordered_by_uncertainty


In [15]:
def find_most_uncertain_parameter_on_I(list_of_matrices,list_of_parameters):
    n = len(list_of_matrices) #this corresponds to the number of parameters
    #Calculate uncertainty by adding up all differences of the min and max curves
    compartment_I = []
    for i in range(n):
        A = np.matrix(np.copy(list_of_matrices[i]))
        v = A.sum(axis=0)
        compartment_I.append(v[0,4])
    #we want a list of all parameters and their uncertainty value
    ordered_by_uncertainty = []
    #if the largest value is recorded, then the value is set to -1, to get the second biggest value.
    for i in range(n):
        k = np.argmax(compartment_I)
        value, name = compartment_I[k]/183, list_of_parameters[k]
        ordered_by_uncertainty.append([name, value])
        #the second largest. All values in these matrices are positive, so we simply put it to -1
        compartment_I[k] = -1
    return ordered_by_uncertainty

# Plot all curves and calculate the sensitivity

## You need a folder with the name: "perturbed_compartment_plots" in your path

In [16]:
def plot_all_uncertainty_with_params(original_curves, params, string_params, n_perturb, perturb_scale):

    list_of_maxMinusmin = []
    for i in range(len(params)):
        single_perturbed_sol, parameter_perturbed = perturb_single_param_2(n_perturb, params, perturb_scale, i)
        S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_perturbed_sol)
        list_of_maxMinusmin.append(S_max-S_min)
        plot_compartments_2(original_curves, parameter_perturbed, single_perturbed_sol, string_params, i)
        print('Done with parameter ' + string_params[i])

    #calculate the min/ max diviation quantitatively
    print('Ordered from largest to smallest influence overall')
    all_params_ordered_by_uncertainty = find_most_uncertain_parameter(list_of_maxMinusmin, string_params)
    df = pd.DataFrame(all_params_ordered_by_uncertainty, columns=['param_name', 'value'])
    df.to_csv('all_params_ordered_by_uncertainty.csv')
    print(df)

    #calculate the min/max diviation of I quantitatively
    print('Ordered from largest to smallest influence on I')
    all_params_ordered_by_uncertainty = find_most_uncertain_parameter_on_I(list_of_maxMinusmin, string_params)
    df = pd.DataFrame(all_params_ordered_by_uncertainty, columns=['param_name', 'value'])
    df.to_csv('all_params_ordered_by_uncertainty_influence_on_I.csv')
    print(df)


    #add plot: all params are perturbed
    all_perturbed = perturb_params(n_perturb, params, perturb_scale)
    plot_compartments_all(original_curves,  all_perturbed, ['all'],0)
    print('All parameters were perturbed. The computation is done.')

# Perform perturbation and uncertainty analysis

The calculations and plots are carried out automatically.

The plots are saved in "perturbed_compartment_plots".
The results of the uncertainty analysis is stored in two files:
1. all_params_ordered_by_uncertainty.csv
2. all_params_ordered_by_uncertainty_influence_on_I.csv

In [ ]:
plot_all_uncertainty_with_params(original, params, string_params,10, 0.05)


Done with parameter β



Done with parameter r_1



Done with parameter r_2



Done with parameter γ


# Specific range perturbation from Boxplots
## You need a folder with the name: "perturb_specific_range" in your path

In [ ]:
def perturb_single_param_specific_range(n, original_params, upper_bound, lower_bound, index):

    mesh = np.linspace(upper_bound, lower_bound, n)
    solutions = []
    parameter_diviations = []
    ts = np.linspace(0,183, 183)

    for i in tqdm(range(n)):
        perturbed_params = np.copy(original_params)
        perturbed_params[index] = mesh[i]
        parameter_diviations.append(mesh[i])
        β,r_1,r_2,γ,δ,ψ,α,ϕ,χ,λ,η,a,ρ1,σ1,τ1,ρ2,σ2,τ2,ρ3,σ3,τ3,E,Q_1,Q_2,I,V,U,D,R = perturbed_params

        S = 70000000 - E - Q_1 - Q_2 - I - V
        L = 70000000 - D - R - H - U

        x0 = [S, E, Q_1, Q_2, I, V,    #epidemics
              L, U, D, R, H]   #opinion

        #define the function anew:
        def f_loc(x, t):
            #times of new restrictions
            timeslot_1 = 589 - 549
            timeslot_2 = timeslot_1 + 689-589
            timeslot_3 = timeslot_1+timeslot_2 + 703-689

            if t<(timeslot_1):
                ρ = ρ1
                σ = σ1
                τ = τ1
            elif t>=timeslot_1 and t < timeslot_2:
                ρ = ρ2
                σ = σ2
                τ = τ2
            else:
                ρ = ρ3
                σ = σ3
                τ = τ3

            S,E,Q_1,Q_2,I,V,    L,U,D,R,H = x

            N = S + E + Q_1 + Q_2 + I + V

            Q = Q_1 + Q_2

            R_G = a*R/(1+a*R)

            η_1 = η * 0.897 * (S/(S+V))
            η_2 = η * 0.103
            η_3 = η * 0.897 * (V/(S+V))

            dS = -β * I/(N-Q) * S - ψ*( 1 - (R + τ*(U))/N + D/N ) * S - ρ * (1- (R + τ * (U))/N) * S + η_1 * Q_1
            dE = β * I/(N-Q) * S + β * δ * I/(N-Q) * V - ρ * ( 1- (R + τ * (U))/N) * E - γ * E
            dQ_1 = ρ * ( 1- (R + τ * (U))/N) * S + ρ * ( 1- (R + τ * (U))/N) * E + ρ * ( 1- (R + τ * (U))/N) * V - η_1 * Q_1 - η_2 * Q_1 - η_3 * Q_1
            dQ_2 = σ * ( 1- R/N) * I + η_2 * Q_1 - r_2 * Q_2
            dI = γ * E - σ * ( 1- R/N) * I - r_1 * I
            dV = r_1 * I + r_2 * Q_2 + η_3 * Q_1 + ψ * ( 1- (R + τ * (U))/N + D/N) * S - β * δ * I/(N-Q) * V - ρ * ( 1- (R + τ * (U))/N) * V

            dL = -α * R/N * L*(1-(I+Q_2)/N) - α * U/N * L - α * D/N * L*(1+(I+Q_2)/N) + ϕ * R + ϕ * H

            dU = α * U/N * L-  χ* R_G/N * U *(1-(I+Q_2)/N) - χ * D/N * U*(1+(I+Q_2)/N) - λ * U
            dD = α * D/N * L*(1+(I+Q_2)/N) + χ * D/N * U*(1+(I+Q_2)/N) - λ * D
            dR = α * R/N * L* (1-(I+Q_2)/N) + χ * R_G/N * U*(1-(I+Q_2)/N) - ϕ * R
            dH = λ * D + λ * U - ϕ * H

            return [dS,dE,dQ_1,dQ_2,dI,dV,dL,dU,dD,dR,dH]



        solutions.append(odeint(f_loc, x0, ts))

    return solutions, parameter_diviations

In [ ]:
#Specific cases: adjust where it should be saved!
def plot_specific_uncertainty_with_bounds(original_curves, params, string_params, n_perturb, upper_bound, lower_bound, perturb_index):

    single_perturbed_sol, parameter_perturbed = perturb_single_param_specific_range(n_perturb, params, upper_bound, lower_bound, perturb_index)
    plot_compartments_2(original_curves, parameter_perturbed, single_perturbed_sol, string_params, perturb_index, True)
    print('Done with parameter ' + string_params[perturb_index])
    S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_perturbed_sol)
    return S_max - S_min


# Calculate and plot specific range perturbations

The calculations and plots are carried out automatically.
The plots are saved in "perturb_specific_range".
The results of the sensitivity analysis is stored the file: specific_bounds_parameter_ordered_by_sensitivity.csv.

In [ ]:
#params = np.array([β,r_1,r_2,γ,δ,ψ,α,ϕ,χ,λ,η,a,ρ1,σ1,τ1,ρ2,σ2,τ2,ρ3,σ3,τ3,E,Q_1,Q_2,I,V,U,D,R])
min_max_liste = []
n_perturbations = 50
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=0.0003, lower_bound= 0.00016, perturb_index=12)) #rho 1
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=0.15, lower_bound= 0.09, perturb_index=13)) # σ 1
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=0.90, lower_bound= 0.18, perturb_index=14)) #tau 1
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=0.00125, lower_bound= 0.0009, perturb_index=15)) #rho 2
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=0.065, lower_bound= 0.03, perturb_index=16)) #σ 2
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=0.98, lower_bound= 0.20, perturb_index=17)) #tau 2
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=0.0025, lower_bound= 0, perturb_index=18)) #rho 3
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=0.19, lower_bound= 0.16, perturb_index=19)) #sigma3
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=1, lower_bound= 0.35, perturb_index=20)) #tau 3


min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=8500, lower_bound= 1800, perturb_index=21)) #E
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=8600, lower_bound= 2700, perturb_index=22)) #Q_1
min_max_liste.append(plot_specific_uncertainty_with_bounds(original, params, string_params,n_perturbations, upper_bound=8200, lower_bound= 1800, perturb_index=24)) #I

print('Ordered from largest to smallest')
string_params_short = ['ρ1','σ1','τ1','ρ2','σ2','τ2','ρ3','σ3','τ4','E','Q_1','I']
ordered_uncertainty_params = find_most_uncertain_parameter(min_max_liste, string_params_short)
print(ordered_uncertainty_params)
df = pd.DataFrame(ordered_uncertainty_params, columns=['param_name', 'value'])
print(df)
df.to_csv('specific_bounds_parameter_ordered_by_uncertainty.csv')

# Load sensitivity results if needed:

In [ ]:
df = pd.read_csv('all_params_ordered_by_uncertainty.csv', usecols=['param_name', 'value'])
print('all_params_ordered_by_uncertainty.csv \n' )
print(df)

df3 = pd.read_csv('all_params_ordered_by_uncertainty_influence_on_I.csv', usecols=['param_name', 'value'])
print('all_params_ordered_by_uncertainty_influence_on_I.csv')
print(df3)

df2 = pd.read_csv('specific_bounds_parameter_ordered_by_uncertainty.csv', usecols=['param_name', 'value'])
print('specific_bounds_parameter_ordered_by_uncertainty')
print(df2)